# Hello World: Remote Data Ingestion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/01_quickstart/01_hello_world.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/01_quickstart/01_hello_world.ipynb)

## Business Scenario

You have a raw data file at a public URL and you need to ingest it into
your lakehouse with quality validation — without downloading files, setting up
databases, or writing custom ETL code.

## Value Proposition

- Point at any remote URL and get a governed, validated table in seconds
- Define rules once in a contract; they apply every run
- Switch between Python dict contracts (prototyping) and YAML files (production) with zero code change

---

## Goals

1. Run LakeLogic against a remote CSV with an **in-memory contract** (Python dict)
2. Repeat using a **YAML file contract** — same result, production-ready
3. Inspect `result.raw`, `result.good`, and `result.bad`


## Setup

In [ ]:
import importlib.util, os, sys
from pathlib import Path

if importlib.util.find_spec("lakelogic") is None:
    import subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "lakelogic", "-q"], check=True)
    print("lakelogic installed.")
else:
    print("lakelogic ready.")

if "google.colab" in sys.modules:
    repo = Path("/content/LakeLogic")
    if not repo.exists():
        import subprocess
        subprocess.run(["git", "clone", "--quiet",
                        "https://github.com/lakelogic/LakeLogic.git", str(repo)], check=True)
    os.chdir(repo / "examples" / "01_quickstart")
    print(f"Working directory: {Path.cwd()}")

from lakelogic import DataProcessor

REMOTE_URL = (
    "https://raw.githubusercontent.com/lakelogic/LakeLogic/main/"
    "examples/01_quickstart/files/excel/data/employees.csv"
)
print(f"Source URL: {REMOTE_URL}")
print("Setup complete.")


## How It Works

LakeLogic reads the source URL, applies the contract's quality rules row-by-row,
and returns a `ValidationResult` with three DataFrames:

| Attribute | Contents |
|---|---|
| `result.raw` | Every row exactly as read from the source |
| `result.good` | Rows that passed all quality rules — safe for analytics |
| `result.bad` | Rows that failed — quarantined with an error reason column |

### Contract formats

LakeLogic accepts contracts two ways — both produce identical results:

| Format | Best for | How to pass |
|---|---|---|
| Python `dict` | Prototyping, dynamic generation, notebooks | `DataProcessor(contract=my_dict)` |
| YAML file | Production, Git versioning, team governance | `DataProcessor(contract="path/to.yaml")` |

### Quality rule applied

```yaml
quality:
  row_rules:
    - name: Valid Email
      sql: "email LIKE '%@%'"   # any row without @ in email → quarantined
```


## 1. Run via In-Memory Contract (Python Dict)

Best for prototyping. Define and run the contract in a single cell — no files needed.


In [ ]:
contract_dict = {
    "version": "1.0.0",
    "dataset": "remote_employees",
    "source": {"type": "landing"},
    "quality": {
        "row_rules": [
            {"name": "Valid Email", "sql": "email LIKE '%@%'"}
        ]
    }
}

processor = DataProcessor(contract=contract_dict)
result    = processor.run_source(REMOTE_URL)

print(f"Source rows : {len(result.raw)}")
print(f"Good rows   : {len(result.good)}")
print(f"Bad rows    : {len(result.bad)}")

print("\nGOOD DATA (passed quality rules):")
display(result.good)


## 2. Run via YAML File Contract

Best for production. The contract lives in a `.yaml` file tracked in Git;
the code never changes, only the contract does.


In [ ]:
import yaml

yaml_path = Path("users_contract_remote.yaml")
yaml_path.write_text(yaml.dump(contract_dict), encoding="utf-8")

processor_prod = DataProcessor(contract=str(yaml_path))
result_prod    = processor_prod.run_source(REMOTE_URL)

print(f"YAML contract — Good: {len(result_prod.good)}, Bad: {len(result_prod.bad)}")
print("BAD DATA (quarantined):")
display(result_prod.bad)


---

## Summary

| Step | What happened |
|---|---|
| Source | Remote CSV read directly — no download needed |
| Quality rule | `email LIKE '%@%'` filtered rows without a valid email |
| `result.good` | Rows safe for analytics |
| `result.bad` | Rows quarantined with error reason |
| YAML vs dict | Both produced identical results |

### What LakeLogic did automatically

- Fetched the remote CSV without local file I/O
- Applied the quality SQL rule row-by-row
- Split results into `good` / `bad` with zero custom code
- Added `_lakelogic_processed_at` and `_lakelogic_run_id` audit columns

---

## Next Steps — Try It Yourself

### 1. Change the quality rules

```python
contract_dict = {
    "version": "1.0.0",
    "dataset": "remote_employees",
    "source": {"type": "landing"},
    "quality": {
        "row_rules": [
            {"name": "Valid Email",  "sql": "email LIKE '%@%'"},
            {"name": "Has Name",     "sql": "name IS NOT NULL AND name != ''"},
            {"name": "Dept Not Null","sql": "department IS NOT NULL"},  # add a rule
        ]
    }
}
```

**Ideas:**
- Point `REMOTE_URL` at your own CSV hosted on GitHub or S3
- Add `accepted_values` rules: `"department IN ('Engineering', 'Sales')"` 
- Add `materialization` to write `result.good` directly to a Parquet file

### 2. Key contract knobs

| What to change | Where in contract | Effect |
|---|---|---|
| Quality rules | `quality.row_rules` | Filter rows into good / bad |
| Source type | `source.type` | `landing` (files/URLs), `table` (dict/rows) |
| Write output | `materialization.target_path` + `format` | Persist `good` rows to Parquet/CSV/Delta |
| Schema enforcement | `model.fields` | Validate column types and required fields |

### 3. Explore related quickstarts

- [`02_database_governance.ipynb`](02_database_governance.ipynb) — same quality-gate pattern applied to a SQLite database
- [`../02_core_patterns/scd2_dimension/`](../02_core_patterns/scd2_dimension/playbook.ipynb) — add SCD2 history tracking to any pipeline
- [`../02_core_patterns/soft_delete/`](../02_core_patterns/soft_delete/soft_delete_pattern.ipynb) — flag deletes instead of removing rows
